In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

def extract_item(soup, target_label):
    """
    指定した項目名（target_label）に対応する値を取得する関数。
    
    Args:
        soup (BeautifulSoup): BeautifulSoupオブジェクト
        target_label (str): 探索する項目名（例: "就業場所"）

    Returns:
        list: 項目名に対応する値のリスト
    """
    rows = soup.find_all("tr", class_="border_new")
    extracted_values = []
    for row in rows:
        label = row.find("td", class_="fb")
        if label and target_label in label.text.strip():
            value = row.find("td").find_next("div")
            if value:
                extracted_values.append(value.text.strip())
    return extracted_values

# データを取得
url = 'https://www.hellowork.mhlw.go.jp/kensaku/GECA110020.do?screenId=GECA110020&action=searchBtn&jgshNo=XPvJ6Jl7g8FnHxrFBQOqRQ%3D%3D&iNFTeikyoRiyoDtiID=&ksNo='
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# 各項目を取得
job_types = extract_item(soup, "求人区分")
work_locations = extract_item(soup, "就業場所")
job_descriptions = extract_item(soup, "仕事の内容")
ages = extract_item(soup, "年齢")

# データが一致しているか確認
if len(job_types) != len(work_locations) or len(work_locations) != len(job_descriptions) or len(job_descriptions) != len(ages):
    print("Warning: 項目数が一致しません。データの対応関係が崩れている可能性があります。")
else:
    # SQLiteデータベースに格納
    conn = sqlite3.connect("job_data.db")
    cursor = conn.cursor()

    # 既存のテーブルを削除
    cursor.execute("DROP TABLE IF EXISTS job_info")

    # テーブルを作成（`job_description`と`age`カラムを追加）
    cursor.execute("""
    CREATE TABLE job_info (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        job_type TEXT,
        work_location TEXT,
        job_description TEXT,
        age TEXT
    )
    """)

    # データを挿入
    for job_type, work_location, job_description, age in zip(job_types, work_locations, job_descriptions, ages):
        cursor.execute("INSERT INTO job_info (job_type, work_location, job_description, age) VALUES (?, ?, ?, ?)", 
                       (job_type, work_location, job_description, age))
        
        # 1秒間隔で挿入する
        time.sleep(1)
    
    conn.commit()
    conn.close()

    print("データをデータベースに格納しました。")


データをデータベースに格納しました。


In [4]:
# データベースに接続
conn = sqlite3.connect('job_data.db')

# カーソルを作成
cursor = conn.cursor()

# データを選択
cursor.execute("SELECT * FROM job_info")

# 結果をフェッチ
results = cursor.fetchall()

# 結果を表示
for row in results:
    print(f"ID: {row[0]}, Job Type: {row[1]}, Work Location: {row[2]}, Job Description: {row[3]}, Age: {row[4]}")

# 接続を閉じる
conn.close()


ID: 1, Job Type: パート, Work Location: 千葉県東金市, Job Description: ＊老人ホームの厨房で、食事を提供するお仕事。出来上がった料理の盛り付け、専用カートでの配膳、, Age: 不問
ID: 2, Job Type: パート, Work Location: 千葉県富津市, Job Description: 富津市上にある老人ホーム厨房で、食事を提供するお仕事。メニューに沿った調理、仕込み、出来上がった料理の盛り付け、果物カットなど、専用カートでの配膳、使用した食器や, Age: 不問
ID: 3, Job Type: パート, Work Location: 千葉県富津市, Job Description: 富津市上にある老人ホーム厨房で、食事を提供するお仕事。出来上がった料理の盛り付け、専用カートでの配膳、使用した食器や器具の洗浄などを行います。, Age: 不問
ID: 4, Job Type: パート, Work Location: 千葉県富津市, Job Description: 老人ホームにて調理補助業務を行っていただきます。【主な業務】・調理補助（仕込み、盛付、洗浄、配膳等）, Age: 不問
ID: 5, Job Type: パート, Work Location: 千葉県習志野市, Job Description: 共立習志野台病院において、調理補助業務を行っていただきます。【主な業務】, Age: 不問
ID: 6, Job Type: パート, Work Location: 千葉県習志野市, Job Description: 病院において調理業務を行っていただきます。調理業務、盛り付け、洗浄, Age: 不問
ID: 7, Job Type: パート, Work Location: 千葉県柏市, Job Description: 老人ホームで調理補助業務を行っていただきます。・配膳・盛り付け, Age: 不問
ID: 8, Job Type: パート, Work Location: 千葉県浦安市, Job Description: 東京ベイ・浦安市川医療センターにおいて、洗浄業務を行っていただきます。, Age: 不問
ID: 9, Job Type: フルタイム, Work Location: 千

In [5]:
# データベースに接続
conn = sqlite3.connect('job_data.db')

# カーソルを作成
cursor = conn.cursor()

# `work_location` が同じものをカウントして多い順に並べるクエリ
cursor.execute("""
SELECT work_location, COUNT(*) AS location_count
FROM job_info
GROUP BY work_location
ORDER BY location_count DESC
""")

# 結果をフェッチ
results = cursor.fetchall()

# 結果を表示
print("Work Location Count (Descending Order):")
for row in results:
    print(f"Location: {row[0]}, Count: {row[1]}")

# 接続を閉じる
conn.close()

Work Location Count (Descending Order):
Location: 千葉県柏市, Count: 7
Location: 千葉県香取市, Count: 5
Location: 千葉県香取郡東庄町, Count: 3
Location: 千葉県市原市, Count: 3
Location: 千葉県富津市, Count: 3
Location: 千葉県千葉市稲毛区, Count: 3
Location: 千葉県習志野市, Count: 2
Location: 千葉県市川市, Count: 2
Location: 千葉県浦安市, Count: 1
Location: 千葉県東金市, Count: 1
